In [1]:
!pip install pinecone
!pip install langchain-pinecone

  Using cached langchain_core-0.3.25-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.25-py3-none-any.whl (411 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.43
    Uninstalling langchain-core-0.2.43:
      Successfully uninstalled langchain-core-0.2.43
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.2.17 requires langchain-core<0.3.0,>=0.2.43, but you have langchain-core 0.3.25 which is incompatible.
langchain-community 0.2.10 requires langchain-core<0.3.0,>=0.2.23, but you have langchain-core 0.3.25 which is incompatible.
langchain-groq 0.1.9 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.3.25 which is incompatible.
langchain-text-splitters 0.2.2 requires langchain-core<0.3.0,>=0.2.10, but you have langchain-core 0.3.25 which is incompatible.


In [2]:
!pip install langchain_groq

  Using cached langchain_core-0.2.43-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.43-py3-none-any.whl (397 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 0.1.2 requires langchain-core<0.4.0,>=0.3.15, but you have langchain-core 0.2.43 which is incompatible.
langchain-pinecone 0.2.0 requires langchain-core<0.4,>=0.3, but you have langchain-core 0.2.43 which is incompatible.


In [39]:
import os
import time
from dotenv import load_dotenv
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from pinecone import Pinecone
from pinecone import ServerlessSpec
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


True

In [4]:
from langchain.chains.question_answering import load_qa_chain
from langchain_pinecone import PineconeVectorStore
from langchain_groq import ChatGroq

In [6]:
api_key = os.getenv("PINECONE_API_KEY")

In [7]:
##Read the Document 
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents


In [8]:
doc=read_doc('uploads/')
doc
len(doc)

1

In [9]:
# cHUNKING Of The Data
def chunk_data(docs,chunk_size=800,chunk_overlap=80):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunked_docs=text_splitter.split_documents(docs)
    return chunked_docs

In [10]:
chunked_docs=chunk_data(doc)
chunked_docs

[Document(metadata={'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf', 'page': 0}, page_content='SL.NO. CANDIDATE NAME GENDER COURSE STREAM INSTITUTE NAME EMAIL ADDRESS CONTACT NO.\n1 SUBRAT SETHY Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE subratsethy017@gmail.com 8658132036\n2 ABHIJIT SAMANT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE lukshi007@gmail.com 7735099393\n3 SHUBHAM PRADHAN Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE shubhampradhan813@gmail.com 8770367753\n4 ABHIJIT MANSINGH Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE abhijitmsing08@gmail.com 8926236278\n5 AYAN KANUNGO Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE aayankanungo@gmail.com 9437851770\n6 ANSUMAN SIMANTA SEKHAR BHUJABALA Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ansumanbhujabal1@gmail.com 8260964834'),
 Document(metadata={'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 

In [11]:
len(chunked_docs)

3

In [12]:
embeddings = HuggingFaceEmbeddings()

In [13]:
vectors=embeddings.embed_documents("How is me ?")
len(vectors)

11

In [40]:

pc = Pinecone(api_key=api_key)


In [41]:

index_name="budget"  

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
print(existing_indexes)

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)
        print("already")
index = pc.Index(index_name)

['budget']


In [35]:
index = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=embeddings
    )

In [29]:
llm = ChatGroq(
        model="llama-3.1-70b-versatile",
        temperature=0
    )
chain=load_qa_chain(llm,chain_type="stuff")


In [30]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [31]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [32]:
our_query = "Tell me about the document "
answer = retrieve_answers(our_query)
print(answer)

[Document(id='1e94b118-d725-4a6b-a3e0-284ccad33f1a', metadata={'page': 0.0, 'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf'}, page_content='SL.NO. CANDIDATE NAME GENDER COURSE STREAM INSTITUTE NAME EMAIL ADDRESS CONTACT NO.\n1 SUBRAT SETHY Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE subratsethy017@gmail.com 8658132036\n2 ABHIJIT SAMANT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE lukshi007@gmail.com 7735099393\n3 SHUBHAM PRADHAN Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE shubhampradhan813@gmail.com 8770367753\n4 ABHIJIT MANSINGH Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE abhijitmsing08@gmail.com 8926236278\n5 AYAN KANUNGO Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE aayankanungo@gmail.com 9437851770\n6 ANSUMAN SIMANTA SEKHAR BHUJABALA Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ansumanbhujabal1@gmail.com 8260964834\n7 IPSITA NAYAK Female B.E. / B.TECH CSE PARALA M

The document appears to be a list of students from Parala Maharaja Engineering College. The list includes the following information about each student:

1. Serial number (SL.NO)
2. Candidate name
3. Gender
4. Course (B.E. / B.TECH)
5. Stream (CSE, EACE, ETC)
6. Institute name (Parala Maharaja Engineering College)
7. Email address
8. Contact number

There are 18 students listed in the document, with 12 males and 6 females. The majority of the students (11) are from the CSE (Computer Science and Engineering) stream, while 6 are from the ETC (Electronics and Telecommunication Engineering) stream, and 1 is from the EACE (Electronics and Communication Engineering) stream.


In [33]:
our_query = "Tell me  about the student ANSUMAN  "
answer = retrieve_answers(our_query)
print(answer)

[Document(id='1e94b118-d725-4a6b-a3e0-284ccad33f1a', metadata={'page': 0.0, 'source': 'uploads/Eligible Candidates List_Keywords Studios_PMEC_2025 Batch.pdf'}, page_content='SL.NO. CANDIDATE NAME GENDER COURSE STREAM INSTITUTE NAME EMAIL ADDRESS CONTACT NO.\n1 SUBRAT SETHY Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE subratsethy017@gmail.com 8658132036\n2 ABHIJIT SAMANT Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE lukshi007@gmail.com 7735099393\n3 SHUBHAM PRADHAN Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE shubhampradhan813@gmail.com 8770367753\n4 ABHIJIT MANSINGH Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE abhijitmsing08@gmail.com 8926236278\n5 AYAN KANUNGO Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE aayankanungo@gmail.com 9437851770\n6 ANSUMAN SIMANTA SEKHAR BHUJABALA Male B.E. / B.TECH CSE PARALA MAHARAJA ENGINEERING COLLEGE ansumanbhujabal1@gmail.com 8260964834\n7 IPSITA NAYAK Female B.E. / B.TECH CSE PARALA M

According to the provided information, ANSUMAN SIMANTA SEKHAR BHUJABALA is a male student. He is pursuing a B.E. / B.TECH degree in the Computer Science and Engineering (CSE) stream at PARALA MAHARAJA ENGINEERING COLLEGE. His contact details are:

Email: ansumanbhujabal1@gmail.com
Contact Number: 8260964834
